In [2]:
import pandas as pd
import useful_functions as use
import psycopg2
import pyodbc
import datetime
from tqdm.notebook import tqdm
tqdm.pandas()
from fuzzywuzzy import fuzz

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
import pyodbc
conn = psycopg2.connect(
    host='oneview-prd-content-aurora-cluster.cluster-cxgp9osuwqi3.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='VFauC8cRXqKapC8A')

In [4]:
username = 'vigrose'
password_edw = 'Hufflepuff~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)

In [5]:
ov_me_sql = '''
    SELECT medical_education_number,type FROM ONEVIEW.physician
    '''
ov_me = pd.read_sql_query(ov_me_sql, conn)

In [6]:
ME_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    AND
    P.ACTIVE_IND = 'Y'
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [7]:
ME_QUERY = \
    f"""
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ME,
    P.UPDATE_TMSTP,
    P.ACTIVE_IND
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 18
    """
party_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [11]:
ME_QUERY = \
    """
    SELECT
    P.PARTY_ID,
    P.KEY_VAL AS ENTITY_ID
    FROM
    AMAEDW.PARTY_KEY P
    WHERE
    P.KEY_TYPE_ID = 9
    """
entity_ids = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [8]:
NAME_QUERY = \
    """
    SELECT
    FIRST_NM, MIDDLE_NM, LAST_NM, SUFFIX, LABEL_NM, NM_TYPE, PARTY_ID
    FROM
    AMAEDW.PERSON_NM
    WHERE
    THRU_DT IS NULL
    AND
    NM_TYPE='LN'
    """
NAMES = pd.read_sql(con=AMAEDW, sql=NAME_QUERY)

In [9]:
PERSON_QUERY = \
    """
    SELECT
    BIRTH_DT, BIRTH_CITY_NM, BIRTH_STATE_ID, BIRTH_COUNTRY_ID, GENDER_CD, MORTALITY_STS_CD, DEATH_DT, MED_EDU_NBR, PARTY_ID
    FROM
    AMAEDW.PERSON
    WHERE
    ACTIVE_IND = 'Y'
    """
PERSONS = pd.read_sql(con=AMAEDW, sql=PERSON_QUERY)

In [14]:
race = pd.read_csv('../../Data/MasterfileCore/PhysicianRaceEthnicity_2_2022-05-02.csv')
race.medical_education_number = use.fix_me(race.medical_education_number)

In [12]:
IDS = pd.merge(entity_ids, party_ids, on='PARTY_ID')
test = pd.merge(IDS, ov_me, left_on='ME', right_on='medical_education_number', how='right')
test_cont = pd.merge(test, NAMES, on='PARTY_ID',how='left').drop_duplicates()
ALL = pd.merge(test_cont, PERSONS, on='PARTY_ID',how='left').drop_duplicates()

In [15]:
ALL = pd.merge(ALL, race, on='medical_education_number', how='left')

In [117]:
ALL.GENDER_CD.unique()

array(['F', 'M', None, nan, 'N'], dtype=object)

In [27]:
test = ['(', ')', ',' ,'/', '_']

In [121]:
len('01676710236')

11

In [125]:
ALL.BIRTH_DT.dtypes

dtype('O')

In [127]:
ALL.BIRTH_DT = pd.to_datetime(ALL.BIRTH_DT)

In [130]:
ALL['BIRTH_YEAR'] = [x.year for x in ALL.BIRTH_DT]

In [26]:
ALL[~(ALL.FIRST_NM == 'None')]

,PARTY_ID,ENTITY_ID,ME,UPDATE_TMSTP,ACTIVE_IND,medical_education_number,type,FIRST_NM,MIDDLE_NM,LAST_NM,...,NM_TYPE,BIRTH_DT,BIRTH_CITY_NM,BIRTH_STATE_ID,BIRTH_COUNTRY_ID,GENDER_CD,MORTALITY_STS_CD,DEATH_DT,MED_EDU_NBR,race_ethnicity
0,7556003,46715925,03575140415,2014-07-26 18:56:16,Y,03575140415,Physician,Veena,,Ramgopal,...,LN,1986-04-23,Far Rockway,222.0,6705.0,F,None,None,03575140415,Asian
1,7556004,46715938,03575071561,2014-07-26 18:56:16,Y,03575071561,Physician,Geena,K,Varghese,...,LN,1982-08-10,New York,222.0,6705.0,F,None,None,03575071561,Asian
2,7556008,46715990,00376160084,2014-07-26 18:56:16,Y,00376160084,Physician,Sheena,,Dhadwal,...,LN,1989-07-13,Long Beach,232.0,6705.0,F,None,None,00376160084,Asian
3,7556009,46716003,00576160461,2014-07-26 18:56:16,Y,00576160461,Resident,Tejas,Y,Chhaya,...,LN,1990-03-17,Covina,232.0,6705.0,M,None,None,00576160461,Asian
4,7556011,46716029,01602180081,2014-07-26 18:56:16,Y,01602180081,Resident,Lukas,Heinrich,Matern,...,LN,1992-09-02,FREIBURG IM BREISGAU,NaN,6133.0,M,None,None,01602180081,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1420062,16848762,71426732,05575210980,2022-05-20 08:31:12,Y,05575210980,Resident,Brooke,,Kania,...,LN,1991-10-22,None,NaN,6942.0,F,None,None,05575210980,NaN
1420063,16848783,71427005,01176210849,2022-05-20 08:31:12,Y,01176210849,Resident,Jonathan,James,Heyl,...,LN,1995-02-26,None,NaN,6942.0,M,None,None,01176210849,NaN
1420064,16848867,71428097,03475220543,2022-05-20 08:31:12,Y,03475220543,Resident,Megan,,Miranda,...,LN,1994-05-06,None,NaN,6942.0,F,None,None,03475220543,NaN
1420065,16850356,71435780,02075170699,2022-05-20 20:38:13,Y,02075170699,Physician,Autumn,,Hammonds,...,LN,1983-03-01,None,NaN,6942.0,F,None,None,02075170699,NaN


In [16]:
today = datetime.date.today()
types = ['Physician','Student','Resident']
dead_mes = list(ALL[(ALL.MORTALITY_STS_CD=='C')].ME)
all_mes = list(ALL.ME)
us_mes = list(ALL[(ALL.BIRTH_COUNTRY_ID==6705)].ME)
elements = [
    {'data_element': 'Party ID',
     'element':'PARTY_ID',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Entity ID',
     'element':'ENTITY_ID',
    'null': '',
    'universe': all_mes},
    {'data_element': 'First Name',
     'element':'FIRST_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Middle Name',
     'element':'MIDDLE_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Last Name',
     'element':'LAST_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Name Suffix',
     'element':'SUFFIX',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Label Name',
     'element':'LABEL_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Name Type',
     'element':'NM_TYPE',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Person Type',
     'element':'type',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Race/Ethnicity',
     'element':'race_ethnicity',
    'null': 'Unknown',
    'universe': all_mes},
    {'data_element': 'Birth Date',
     'element':'BIRTH_DT',
    'null': datetime.date(1900, 1, 1),
    'universe':all_mes},
    {'data_element': 'Birth City',
     'element':'BIRTH_CITY_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Birth State',
     'element':'BIRTH_STATE_ID',
    'null': '',
    'universe': us_mes},
    {'data_element': 'Birth Country',
     'element':'BIRTH_COUNTRY_ID',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Gender',
     'element':'GENDER_CD',
    'null': '',
    'universe':all_mes},
    {'data_element': 'Death Date',
     'element':'DEATH_DT',
    'null': '',
    'universe':dead_mes},     
]
dict_list = []
for typo in types:
    total = len(ALL[ALL.type==typo])
    for element in elements:
        universe = len(ALL[(ALL.type==typo)&(ALL.ME.isin(element['universe']))])
        complete = len(ALL[(ALL.type==typo)&~(ALL[element['element']].isna())&(ALL.ME.isin(element['universe']))])
        true_complete = len(ALL[(ALL.type==typo)&(ALL[element['element']]!=element['null'])&~(ALL[element['element']].isna())&(ALL.ME.isin(element['universe']))])
        new_dict = {
            'Universe': typo,
            'Data Element': element['data_element'],
            'Complete': complete,
            'Complete and Known': true_complete,
            'Universe Total': universe,
            'Date': today,
            'Measure':'Completeness'
        }
        dict_list.append(new_dict)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [20]:
today = datetime.date.today()
types = ['Physician','Student','Resident']
dead_mes = list(ALL[(ALL.MORTALITY_STS_CD=='C')].ME)
all_mes = list(ALL.ME)
us_mes = list(ALL[(ALL.BIRTH_COUNTRY_ID==6705)].ME)
elements = [
    {'data_element': 'Party ID',
     'element':'PARTY_ID',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Entity ID',
     'element':'ENTITY_ID',
    'null': '',
    'universe': all_mes},
    {'data_element': 'First Name',
     'element':'FIRST_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Middle Name',
     'element':'MIDDLE_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Last Name',
     'element':'LAST_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Name Suffix',
     'element':'SUFFIX',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Label Name',
     'element':'LABEL_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Name Type',
     'element':'NM_TYPE',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Person Type',
     'element':'type',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Race/Ethnicity',
     'element':'race_ethnicity',
    'null': 'Unknown',
    'universe': all_mes},
    {'data_element': 'Birth Date',
     'element':'BIRTH_DT',
    'null': datetime.date(1900, 1, 1),
    'universe':all_mes},
    {'data_element': 'Birth City',
     'element':'BIRTH_CITY_NM',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Birth State',
     'element':'BIRTH_STATE_ID',
    'null': '',
    'universe': us_mes},
    {'data_element': 'Birth Country',
     'element':'BIRTH_COUNTRY_ID',
    'null': '',
    'universe': all_mes},
    {'data_element': 'Gender',
     'element':'GENDER_CD',
    'null': '',
    'universe':all_mes},
    {'data_element': 'Death Date',
     'element':'DEATH_DT',
    'null': '',
    'universe':dead_mes},     
]
dict_list = []
for element in elements:
    universe = len(ALL[(ALL.ME.isin(element['universe']))])
    complete = len(ALL[~(ALL[element['element']].isna())&(ALL.ME.isin(element['universe']))])
    true_complete = len(ALL[(ALL[element['element']]!=element['null'])&~(ALL[element['element']].isna())&(ALL.ME.isin(element['universe']))])
    new_dict = {
            'Data Element': element['data_element'],
            'Complete': complete,
            'Complete and Known': true_complete,
            'Universe Total': universe,
            'Date': today,
            'Measure':'Completeness'
        }
    dict_list.append(new_dict)

In [21]:
pd.DataFrame(dict_list)

,Data Element,Complete,Complete and Known,Universe Total,Date,Measure
0,Party ID,1420067,1420067,1420067,2022-05-23,Completeness
1,Entity ID,1420067,1420067,1420067,2022-05-23,Completeness
2,First Name,1420067,1420067,1420067,2022-05-23,Completeness
3,Middle Name,1420053,1110018,1420067,2022-05-23,Completeness
4,Last Name,1420067,1420067,1420067,2022-05-23,Completeness
5,Name Suffix,41131,41131,1420067,2022-05-23,Completeness
6,Label Name,1420067,1420067,1420067,2022-05-23,Completeness
7,Name Type,1420067,1420067,1420067,2022-05-23,Completeness
8,Person Type,1420067,1420067,1420067,2022-05-23,Completeness
9,Race/Ethnicity,1397283,1148822,1420067,2022-05-23,Completeness


In [35]:
MIDS = ALL[~ALL.MIDDLE_NM.isna()]

In [55]:
ALL.head().LABEL_NM

0    Veena Ramgopal DO                       
1    Geena K Varghese DO                     
2    Sheena Dhadwal DO                       
3    Tejas Y Chhaya DO                       
4    Lukas Heinrich Matern MD                
Name: LABEL_NM, dtype: object

In [56]:
count = 0
validity = []
for row in tqdm(ALL.itertuples()):
    invalid = False
    if ' MD' not in row.LABEL_NM:
        if ' DO' not in row.LABEL_NM:
            invalid = True
            count +=1
    validity.append(invalid)

In [61]:
NO_DEGREE = ALL[validity]
NO_DEGREE[NO_DEGREE.type!='Student']

,PARTY_ID,ENTITY_ID,ME,UPDATE_TMSTP,ACTIVE_IND,medical_education_number,type,FIRST_NM,MIDDLE_NM,LAST_NM,...,NM_TYPE,BIRTH_DT,BIRTH_CITY_NM,BIRTH_STATE_ID,BIRTH_COUNTRY_ID,GENDER_CD,MORTALITY_STS_CD,DEATH_DT,MED_EDU_NBR,race_ethnicity
303516,2231129,8726467,03875820060,2005-12-16 14:49:35,Y,03875820060,Physician,Monte,,Finch,...,LN,1956-08-31,TEXAS CITY,239.0,5874.0,M,None,None,03875820060,Unknown
303871,2231508,8738986,03875940383,2005-12-16 14:49:35,Y,03875940383,Physician,Thea,,Klingberg,...,LN,1965-03-22,NYC,NaN,5874.0,F,None,None,03875940383,Unknown
459606,2453788,11399176,05575900137,2005-12-16 14:49:35,Y,05575900137,Physician,Shelby,,Johnson,...,LN,1956-02-20,WASHINGTON,244.0,5874.0,F,None,None,05575900137,White
462261,2453718,11390986,05575780018,2005-12-16 14:49:35,Y,05575780018,Physician,Edward,,Bailey,...,LN,1947-03-23,USA,NaN,5874.0,M,None,None,05575780018,Unknown
462262,2453719,11391103,05575780107,2005-12-16 14:49:35,Y,05575780107,Physician,James,,Halley,...,LN,1948-10-15,MARIETTA,252.0,5874.0,M,None,None,05575780107,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1363354,15066254,59139366,04775171095,2020-04-02 21:41:18,Y,04775171095,Resident,Maura,,Fitzpatrick,...,LN,1991-12-14,None,NaN,6942.0,F,None,None,04775171095,Unknown
1389479,15897560,64349597,04775201253,2021-01-21 21:17:54,Y,04775201253,Resident,Matthew,,Middler,...,LN,1985-09-01,Johnstown,257.0,6705.0,M,None,None,04775201253,Unknown
1390552,16287549,66868009,03979191072,2021-07-16 20:48:33,Y,03979191072,Resident,Blair,,Brown,...,LN,1990-07-05,Oklahoma City,215.0,6705.0,F,None,None,03979191072,Unknown
1390945,16312691,67100839,04775210805,2021-08-03 20:43:45,Y,04775210805,Resident,Sheena,,Tran,...,LN,1988-02-13,Fountain Valley,232.0,6705.0,F,None,None,04775210805,Unknown


In [43]:
count = 0
validity = []
for row in tqdm(ALL.itertuples()):
    invalid = False
    if 'None ' in row.LABEL_NM:
        invalid = True
        validity.append(invalid)
        continue
    for character in test:
        if character in row.LABEL_NM:
            invalid = True
            count += 1
            break
    validity.append(invalid)

In [39]:
count = 0
validity = []
for row in tqdm(ALL.itertuples()):
    if len(row.LAST_NM) == 1:
        if row.LAST_NM not in ['A','I','U','O','Y']:
            invalid = True
            validity.append(invalid)
            continue
    invalid = False
    for character in test:
        if character in row.LAST_NM:
            invalid = True
            count += 1
            break
    validity.append(invalid)

In [44]:
ALL[validity]

,PARTY_ID,ENTITY_ID,ME,UPDATE_TMSTP,ACTIVE_IND,medical_education_number,type,FIRST_NM,MIDDLE_NM,LAST_NM,...,NM_TYPE,BIRTH_DT,BIRTH_CITY_NM,BIRTH_STATE_ID,BIRTH_COUNTRY_ID,GENDER_CD,MORTALITY_STS_CD,DEATH_DT,MED_EDU_NBR,race_ethnicity
800569,3333621,21197900,55001860231,2005-12-16 14:49:35,Y,55001860231,Physician,Eliyahu,(Eli),Atar,...,LN,1960-04-20,None,NaN,NaN,M,None,None,55001860231,White
817530,3372991,21709749,49617970111,2016-03-04 21:48:54,Y,49617970111,Resident,None,,Munish,...,LN,1972-12-15,None,NaN,6693.0,M,None,None,49617970111,Mixed Race/Ethnicity
837864,3405754,22135720,67203010025,2005-12-16 14:49:35,Y,67203010025,Physician,Sapana,,Mainali (Khanal),...,LN,1976-11-05,None,NaN,6942.0,F,None,None,67203010025,Unknown
857013,3429866,22449605,20903000024,2005-12-16 14:49:35,Y,20903000024,Physician,Lin,Aung (A) Rex,Hu,...,LN,1972-05-10,None,NaN,6942.0,M,None,None,20903000024,Unknown
858425,3432231,22480350,91307940073,2008-02-11 22:08:52,Y,91307940073,Physician,Genady,,Lilevman (Pivovarov),...,LN,1969-07-19,None,NaN,6942.0,M,None,None,91307940073,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1336000,13454990,51840435,02879021475,2018-11-27 20:40:28,Y,02879021475,Physician,Daniel,,"Milam, Iii",...,LN,None,None,NaN,NaN,None,None,None,02879021475,NaN
1337573,13456613,51861534,00376160815,2018-11-27 20:40:28,Y,00376160815,Physician,Daniel,,"Hogan , Jr",...,LN,None,None,NaN,NaN,None,None,None,00376160815,NaN
1338774,13954725,53710628,53908210014,2021-07-02 20:40:02,Y,53908210014,Physician,"Anna,",Melody,Demian,...,LN,1995-09-19,None,NaN,6942.0,F,None,None,53908210014,Unknown
1390015,16143600,65867347,04825240590,2021-04-26 20:39:37,Y,04825240590,Student,Hermoine (Yamyo),,Monie,...,LN,1995-01-31,None,NaN,NaN,F,None,None,04825240590,NaN


In [87]:
username = 'vigrose'
password_edw = 'Hufflepuff~10946'
w = "DSN=MDM; UID={}; PWD={}".format(username, password_edw)
STG = pyodbc.connect(w)

In [26]:
#abms
BOARD_QUERY = \
    f"""
    SELECT
    PARTY_ID_FROM,
    PARTY_ID_TO,
    ABMS_ID
    FROM
    AMAEDW.BRD_CERTIF
    WHERE
    THRU_DT is null
    """
abms = pd.read_sql(con=AMAEDW, sql=BOARD_QUERY)

abms_person_query = '''
    SELECT 
    AP.PKEY_SRC_OBJECT,
    AP.FRST_NM,
    AP.LST_NM,
    AP.MDL_NM,
    AP.FULL_NM,
    AP.SFX_NM,
    AP.BIRTH_YR,
    AP.BIRTH_DT,
    AP.DEATH_DT,
    AP.RAW_FRST_NM,
    AP.RAW_LST_NM,
    AP.RAW_MDL_NM,
    AP.RAW_FULL_NM,
    AP.PLC_OF_BRTH,
    AP.MORT_STS_CD
    FROM 
    STG.C_S_ABMS_PTY AP
    WHERE 
    AP.DELETED_IND IS NULL
    AND
    AP.HUB_STATE_IND = 1
    '''
abms_person = pd.read_sql(con=STG, sql=abms_person_query)

In [43]:
abms_person['ABMS_ID'] = [float(x.split('~')[1]) for x in abms_person.PKEY_SRC_OBJECT]
abms_all = pd.merge(abms, ALL, left_on='PARTY_ID_FROM', right_on='PARTY_ID')
abms_all = pd.merge(abms_person, abms_all, on='ABMS_ID', suffixes=['_ABMS','_EDW'])
abms_all = abms_all.drop_duplicates()

In [ ]:
abms_dict = abms_all.to_dict('records')

In [63]:
dict_list = []
for row in tqdm(abms_dict):
    birth_dt_match = row['BIRTH_DT_ABMS']==row['BIRTH_DT_EDW']
    death_dt_match = row['DEATH_DT_ABMS']==row['DEATH_DT_EDW']
    last_match = row['RAW_LST_NM']==row['LAST_NM']
    if last_match == False:
        score = fuzz.ratio(row['RAW_LST_NM'],row['LAST_NM'])
    else:
        score = 100
    new_dict = {
        'ME' : row['ME'],
        'BIRTH_DT_MATCH' : birth_dt_match,
        'DEATH_DT_MATCH' : death_dt_match,
        'LAST_MATCH' : last_match,
        'ABMS_LAST': row['RAW_LST_NM'],
        'EDW_LAST': row['LAST_NM'],
        'ABMS_NAME': row['RAW_FULL_NM'],
        'EDW_NAME': row['LABEL_NM'],
        'SCORE': score
        
    }
    dict_list.append(new_dict)

In [75]:
dict_list = []
for row in tqdm(abms_dict):
    first_match = row['FRST_NM']==row['RAW_FRST_NM']
    middle_match = row['MDL_NM']==row['RAW_MDL_NM']
    last_match = row['RAW_LST_NM']==row['LST_NM']
    full_match = row['RAW_FULL_NM']==row['FULL_NM']
    new_dict = {
        'ME' : row['ME'],
        'FIRST_MATCH' : first_match,
        'MIDDLE_MATCH' : middle_match,
        'LAST_MATCH' : last_match,
        'FULL_MATCH' : full_match,
        'RAW_NAME': row['RAW_FULL_NM'],
        'NAME': row['FULL_NM'],
        'EDW_NAME': row['LABEL_NM']
        
    }
    dict_list.append(new_dict)

In [76]:
test = pd.DataFrame(dict_list)

In [112]:
test

,ME,FIRST_MATCH,MIDDLE_MATCH,LAST_MATCH,FULL_MATCH,RAW_NAME,NAME,EDW_NAME
0,04705840151,True,True,True,True,Lee Buell Daniel,Lee Buell Daniel,Lee Buell Daniel MD
1,04705840151,True,True,True,True,Lee Buell Daniel,Lee Buell Daniel,Lee Buell Daniel MD
2,03840811076,True,True,True,True,Deborah M. Meesig,Deborah M. Meesig,Deborah Marie Meesig MD
3,03840811076,True,True,True,True,Deborah M. Meesig,Deborah M. Meesig,Deborah Marie Meesig MD
4,03841780131,True,True,True,True,John Gregory Bors,John Gregory Bors,John Gregory Bors MD
...,...,...,...,...,...,...,...,...
1057692,42201156593,True,True,True,True,Nathaniel Melton,Nathaniel Melton,Nathaniel Lee Melton MD
1057693,02416110916,True,True,True,True,Abraham Edan Jaffe,Abraham Edan Jaffe,Abraham Edan Jaffe MD
1057694,03575181766,True,True,True,True,Syed Z. Ali,Syed Z. Ali,Syed Ali DO
1057695,01611982069,True,True,True,True,Davida A. Robinson,Davida A. Robinson,Davida A Robinson MD


In [74]:
# test = pd.DataFrame(dict_list)
hmm = list(test[(~test.LAST_MATCH)&(test.SCORE<60)].ME)

In [41]:
abms_all.columns

Index(['PKEY_SRC_OBJECT', 'FRST_NM', 'LST_NM', 'MDL_NM', 'FULL_NM', 'SFX_NM',
       'BIRTH_YR', 'BIRTH_DT_ABMS', 'DEATH_DT_ABMS', 'RAW_FRST_NM',
       'RAW_LST_NM', 'RAW_MDL_NM', 'RAW_FULL_NM', 'PLC_OF_BRTH', 'MORT_STS_CD',
       'BRD_ID', 'ABMS_ID', 'PARTY_ID_FROM', 'PARTY_ID_TO', 'PARTY_ID',
       'ENTITY_ID', 'ME', 'UPDATE_TMSTP', 'ACTIVE_IND',
       'medical_education_number', 'type', 'FIRST_NM', 'MIDDLE_NM', 'LAST_NM',
       'SUFFIX', 'LABEL_NM', 'NM_TYPE', 'BIRTH_DT_EDW', 'BIRTH_CITY_NM',
       'BIRTH_STATE_ID', 'BIRTH_COUNTRY_ID', 'GENDER_CD', 'MORTALITY_STS_CD',
       'DEATH_DT_EDW', 'MED_EDU_NBR'],
      dtype='object')

In [42]:
abms_all[['BIRTH_DT_ABMS','BIRTH_DT_EDW']]

,BIRTH_DT_ABMS,BIRTH_DT_EDW
0,1958-03-10,1958-03-10
1,1958-03-10,1958-03-10
2,1954-07-11,1954-07-11
3,1954-07-11,1954-07-11
4,1954-07-11,1954-07-11
...,...,...
2883953,1979-01-05,1979-01-05
2883954,1991-06-07,1991-06-07
2883955,1972-09-14,1972-09-14
2883956,1972-09-14,1972-09-14


In [141]:
with_TOP = pd.merge(ALL, ppd[['ME','TOP_CD']], on='ME')

In [143]:
with_TOP.groupby(['type','TOP_CD']).count()

PARTY_ID  ENTITY_ID      ME  UPDATE_TMSTP  ACTIVE_IND  \
type      TOP_CD                                                          
Physician 12            48         48      48            48          48   
          20        815752     815752  815752        815752      815752   
          30         12940      12940   12940         12940       12940   
          40         12214      12214   12214         12214       12214   
          50         12538      12538   12538         12538       12538   
          62          4667       4667    4667          4667        4667   
          71        172302     172302  172302        172302      172302   
          72         19742      19742   19742         19742       19742   
          74          3964       3964    3964          3964        3964   
          75         14902      14902   14902         14902       14902   
          100        92374      92374   92374         92374       92374   
Resident  12        127039     127039  127039        127039      127039   
          20          2462       2462    2462          2462        2462   
          100        16725      16725   16725         16725       16725   
Student   12             3          3       3             3           3   

                  medical_education_number  FIRST_NM  MIDDLE_NM  LAST_NM  \
type      TOP_CD                                                           
Physician 12                            48        48         48       48   
          20                        815752    815752     815745   815752   
          30                         12940     12940      12940    12940   
          40                         12214     12214      12214    12214   
          50                         12538     12538      12538    12538   
          62                          4667      4667       4667     4667   
          71                        172302    172302     172302   172302   
          72                         19742     19742      19742    19742   
          74                          3964      3964       3964     3964   
          75                         14902     14902      14902    14902   
          100                        92374     92374      92368    92374   
Resident  12                        127039    127039     127038   127039   
          20                          2462      2462       2462     2462   
          100                        16725     16725      16725    16725   
Student   12                             3         3          3        3   

                  SUFFIX  ...  NM_TYPE  BIRTH_DT  BIRTH_CITY_NM  \
type      TOP_CD          ...                                     
Physician 12           2  ...       48        48             36   
          20       22859  ...   815752    813704         720484   
          30         614  ...    12940     12936          12843   
          40         372  ...    12214     12208          11267   
          50         442  ...    12538     12533          12352   
          62         142  ...     4667      4664           4483   
          71        9393  ...   172302    171961         171649   
          72         944  ...    19742     19739          19698   
          74          79  ...     3964      3962           3756   
          75         604  ...    14902     14888          14761   
          100       1233  ...    92374     90452          55965   
Resident  12        2417  ...   127039    126061         108581   
          20          55  ...     2462      2462           2227   
          100        294  ...    16725     16725          14314   
Student   12           0  ...        3         3              3   

                  BIRTH_STATE_ID  BIRTH_COUNTRY_ID  GENDER_CD  \
type      TOP_CD                                                
Physician 12                  32                48         47   
          20              534610            805954     814560   
          30               10485             1292

In [69]:
ppd = use.get_ppd()

INFO:useful_functions:PPD file from Mar 05 2022
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (13,14,20,21,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [82]:
# q = "DSN=eprdods; UID={}; PWD={}".format(username, password_edw)
# ODS = pyodbc.connect(q)
iqvia_query = \
        """
        SELECT
        ME,
        FIRST_NAME,
        MIDDLE_NAME,
        LAST_NAME,
        GEN_SUFFIX,
        GENDER,
        PRIMARY_PROF_DESC
        FROM 
        ODS.ODS_IMS_PROFESSIONAL
        WHERE  
        CURRENT_BATCH_FLAG='Y'
        AND
        ME IS NOT NULL
        """
iqvia = pd.read_sql(con=ODS, sql=iqvia_query)
iqvia.head()

,ME,FIRST_NAME,MIDDLE_NAME,LAST_NAME,GEN_SUFFIX,GENDER,ROLE,PRIMARY_PROF_DESC
0,1431104010,JOHN,PAUL,COTTER,None,M,B,Medical Doctors
1,0120108067,MARY,ASHLEY,CAIN,None,F,B,Medical Doctors
2,0481614019,ROBERT,None,GLADNEY,None,M,B,Medical Doctors
3,0180318053,DANIEL,ALAN,EVANS,None,M,B,Medical Doctors
4,0420101001,MARIELSA,None,RABELO- ACEVEDO,None,F,P,Medical Doctors


In [101]:
license_person_query = '''
    SELECT 
    PKEY_SRC_OBJECT,
    FRST_NM,
    LST_NM,
    MDL_NM,
    FULL_NM,
    SFX_NM,
    BIRTH_YR,
    BIRTH_DT,
    DEATH_DT,
    RAW_FRST_NM,
    RAW_LST_NM,
    RAW_MDL_NM,
    RAW_FULL_NM,
    PLC_OF_BRTH,
    GNDR_CD
    FROM 
    STG.C_S_SLN_PTY
    WHERE 
    DELETED_IND IS NULL
    AND
    HUB_STATE_IND = 1
    '''
license_person = pd.read_sql(con=STG, sql=license_person_query)

In [108]:
license_person[~license_person.SFX_NM.isna()]

,PKEY_SRC_OBJECT,FRST_NM,LST_NM,MDL_NM,FULL_NM,SFX_NM,BIRTH_YR,BIRTH_DT,DEATH_DT,RAW_FRST_NM,RAW_LST_NM,RAW_MDL_NM,RAW_FULL_NM,PLC_OF_BRTH,GNDR_CD
109,MI~4301502897~Medical Doctor,Daniel,Beacher,Robert,None,Beache,NaN,None,None,Daniel,Beacher,Robert,None,None,None
119,MI~4301503089~Medical Doctor,Craig,Butler,D,None,II,NaN,None,None,Craig,Butler,D,None,None,None
182,MI~4301503344~Medical Doctor,Miriam,Harewood-Makola,Atira,None,MD,NaN,None,None,Miriam,Harewood-Makola,Atira,None,None,None
189,MI~4301503234~Medical Doctor,Laura,Honch,Jean,None,M.D.,NaN,None,None,Laura,Honch,Jean,None,None,None
206,MI~4301503382~Medical Doctor,Balreet,Kahlon,K,None,MD,NaN,None,None,Balreet,Kahlon,K,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2891232,AK~193806~MED11,Terence,Chun,Ung Hoong,Terence Ung Hoong Chun,,NaN,None,None,Terence,Chun,Ung Hoong,Terence Ung Hoong Chun,None,None
2891233,AK~194106~MED11,Hillel,BenAvi,Ari,Hillel Ari BenAvi,,NaN,None,None,Hillel,BenAvi,Ari,Hillel Ari BenAvi,None,None
2891234,AK~194314~MED11,Christopher,Braga,Cabral,Christopher Cabral Braga,,NaN,None,None,Christopher,Braga,Cabral,Christopher Cabral Braga,None,None
2891325,AK~194064~MED11,Calvin,Wu,,Calvin Wu,,NaN,None,None,Calvin,Wu,,Calvin Wu,None,None


In [148]:
MED_SCHOOL_QUERY = \
    """
    SELECT
    SCH_PARTY_ID,
    STU_PARTY_ID AS PARTY_ID,
    GRAD_DT,
    STS_TYPE_ID,
    DEGREE_CD,
    GRAD_CONF_IND
    FROM
    AMAEDW.SCHOOL_ATT
    WHERE
    THRU_DT is null
    """
med_school = pd.read_sql(con=AMAEDW, sql=MED_SCHOOL_QUERY)

In [150]:
ALL = pd.merge(med_school, ALL, on='PARTY_ID')

In [154]:
ALL = ALL.drop_duplicates()

In [162]:
ALL['GRAD_DT'] = pd.to_datetime(ALL.GRAD_DT, errors='coerce')

In [164]:
ALL['GRADD'] = [x.year for x in ALL.GRAD_DT]

In [168]:
ALL[(ALL.GRADD<2021)&(ALL.type=='Student')]

,SCH_PARTY_ID,PARTY_ID,GRAD_DT,STS_TYPE_ID,DEGREE_CD,GRAD_CONF_IND,ENTITY_ID,ME,UPDATE_TMSTP,ACTIVE_IND,...,BIRTH_DT,BIRTH_CITY_NM,BIRTH_STATE_ID,BIRTH_COUNTRY_ID,GENDER_CD,MORTALITY_STS_CD,DEATH_DT,MED_EDU_NBR,BIRTH_YEAR,GRADD
490,2701305,11103967,2024-05-22,7,MD,N,48465569,03845200903,2016-11-14 21:13:00,Y,...,1993-05-06,PHILADELPHIA,257.0,6705.0,M,None,None,03845200903,1993.0,2024.0
498,2836982,7599556,2024-05-12,8,MD,None,46820042,02407181762,2014-09-12 21:42:47,Y,...,1992-02-04,HO CHI MINH CITY,NaN,6921.0,M,None,None,02407181762,1992.0,2024.0
607,2855135,15257996,2023-05-01,8,DO,None,60507408,01875230146,2020-05-29 21:26:33,Y,...,1993-03-10,None,NaN,6942.0,M,None,None,01875230146,1993.0,2023.0
728,6756014,15272921,2023-05-28,8,DO,None,60612526,04179230114,2020-06-05 21:41:18,Y,...,1992-05-12,None,NaN,6942.0,M,None,None,04179230114,1992.0,2023.0
800,2701293,12801158,2026-05-21,8,MD,None,49948622,02307260568,2018-07-11 20:44:03,Y,...,1994-04-11,SHIRAZ,NaN,7604.0,F,None,None,02307260568,1994.0,2026.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1412983,2848765,15022469,2022-05-30,8,DO,None,58775093,00577221367,2020-03-20 21:06:07,Y,...,1989-12-18,None,NaN,NaN,None,None,None,00577221367,1989.0,2022.0
1413041,2848789,15040520,2023-05-31,8,DO,None,58933875,04177230249,2020-03-26 21:10:06,Y,...,1996-06-06,Coral Springs,198.0,6705.0,M,None,None,04177230249,1996.0,2023.0
1413371,4665015,15094243,2023-06-01,8,DO,None,59290933,03576231147,2020-04-08 21:26:35,Y,...,1997-06-29,None,NaN,5895.0,M,None,None,03576231147,1997.0,2023.0
1413375,2848789,15094677,2023-06-01,8,DO,None,59296575,04177230257,2020-04-08 21:26:35,Y,...,1995-08-09,None,NaN,6942.0,M,None,None,04177230257,1995.0,2023.0


In [46]:
country = pd.read_csv('../../Data/Measurement/COUNTRY.csv')

In [51]:
list(country.COUNTRY_ID.unique())

[5755,
 5756,
 5757,
 5758,
 5759,
 5760,
 5761,
 5762,
 5763,
 5764,
 5765,
 5766,
 5767,
 5768,
 5769,
 5770,
 5771,
 5772,
 5773,
 5774,
 5775,
 5776,
 5777,
 5778,
 5779,
 5780,
 5781,
 5782,
 5783,
 5784,
 5785,
 5786,
 5787,
 5788,
 5789,
 5790,
 5791,
 5792,
 5793,
 5794,
 5795,
 5796,
 5797,
 5798,
 5799,
 5800,
 5801,
 5802,
 5803,
 5804,
 5805,
 5806,
 5807,
 5808,
 5809,
 5810,
 5811,
 5812,
 5813,
 5814,
 5815,
 5816,
 5817,
 5818,
 5819,
 5820,
 5821,
 5822,
 5823,
 5824,
 5825,
 5826,
 5827,
 5828,
 5829,
 5830,
 5831,
 5832,
 5833,
 5834,
 5835,
 5836,
 5837,
 5838,
 5839,
 5840,
 5841,
 5842,
 5843,
 5844,
 5845,
 5846,
 5847,
 5848,
 5849,
 5850,
 5851,
 5852,
 5853,
 5854,
 5855,
 5856,
 5857,
 5858,
 5859,
 5860,
 5861,
 5862,
 5863,
 5864,
 5865,
 5866,
 5867,
 5868,
 5869,
 5870,
 5871,
 5872,
 5873,
 5874,
 5875,
 5876,
 5877,
 5878,
 5879,
 5880,
 5881,
 5882,
 5883,
 5884,
 5885,
 5886,
 5887,
 5888,
 5889,
 5890,
 5891,
 5892,
 5893,
 5894,
 5895,
 5896,
 5897,

In [53]:
test = pd.read_csv('../../Data/Measurement/Test.csv')

In [62]:
country = pd.read_csv('../../Data/Measurement/COUNTRY.csv')

In [88]:
ALL[ALL.BIRTH_COUNTRY_ID==5796]

,PARTY_ID,ENTITY_ID,ME,UPDATE_TMSTP,ACTIVE_IND,medical_education_number,type,FIRST_NM,MIDDLE_NM,LAST_NM,...,NM_TYPE,BIRTH_DT,BIRTH_CITY_NM,BIRTH_STATE_ID,BIRTH_COUNTRY_ID,GENDER_CD,MORTALITY_STS_CD,DEATH_DT,MED_EDU_NBR,race_ethnicity
613828,2667801,13681066,66004650019,2005-12-16 14:49:35,Y,66004650019,Physician,Andreas,Frank,Siraa,...,LN,1938-06-15,MAARTENSDYK,NaN,5796.0,M,None,None,66004650019,White
667295,2744362,13681508,66004800011,2005-12-16 14:49:35,Y,66004800011,Physician,Tjerk,Fridolin,Bury,...,LN,1955-05-04,LEIDEN,NaN,5796.0,M,None,None,66004800011,White


In [77]:
country[country.COUNTRY_ID.isin(ALL.BIRTH_COUNTRY_ID)].to_csv('../../Data/Measurement/countries.csv', index=False)

In [79]:
import pycountry

In [107]:
pycountry.countries.lookup('jersey')

Country(alpha_2='JE', alpha_3='JEY', name='Jersey', numeric='832')

In [ ]:
#license
#dea
#npi
#symphony
#dhc
#datagov

In [90]:
bads = pd.read_csv('../../Data/Measurement/bad_countries.csv')

In [93]:
BOPS = ALL[(ALL.BIRTH_COUNTRY_ID.isin(bads.COUNTRY_ID))&(ALL.BIRTH_COUNTRY_ID!=6942)]

In [99]:
BOPS.groupby(['BIRTH_COUNTRY_ID','BIRTH_CITY_NM']).count().to_csv('../../Data/Measurement/by_city.csv')

In [100]:
BOPS.groupby(['BIRTH_COUNTRY_ID','BIRTH_STATE_ID']).count().to_csv('../../Data/Measurement/by_state.csv')

In [ ]:
added_countries = []
country_dicts = []
for row in BOPS.itertuples():
    if row.BIRTH_COUNTRY_ID not in added_countries:
        added_countries.append(row.BIRTH_COUNTRY_ID)
        new_dict = {
            
        }